## First example: $a=(\vec{b}+c)\cdot(c+2)$

In [5]:
import tensorflow as tf
import numpy as np

In [6]:
print(tf.__version__)

1.15.0


In [14]:
# 首先，创建一个TensorFlow常量
const = tf.constant(2.0, name='const')

# 创建placeholder
# [None,1]表示不确定第一个维度的大小（可以任意大小）
b = tf.placeholder(tf.float32, [None, 1], name='b')

# 创建TensorFlow变量
c = tf.Variable(1.0, dtype=tf.float32, name='c')

# 创建operation
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a')

# 1. 定义init operation
init_op = tf.global_variables_initializer()

# session
with tf.Session() as sess:
    # 2. 运行init operation
    sess.run(init_op)
    
    # 计算
    a_out = sess.run(a, feed_dict={b: np.arange(0, 10)[:, np.newaxis]})
    print(a_out)

[[ 3.]
 [ 6.]
 [ 9.]
 [12.]
 [15.]
 [18.]
 [21.]
 [24.]
 [27.]
 [30.]]
